http://web.mta.info/developers/turnstile.html
https://groups.google.com/forum/#!searchin/mtadeveloperresources/turnstile/mtadeveloperresources/tynsUKSJtjk/BIdNU_kdKnwJ

C/A      = Control Area (A002)
UNIT     = Remote Unit for a station (R051)
SCP      = Subunit Channel Position represents an specific address for a device
           (02-00-00)
STATION  = Represents the station name the device is located at
LINENAME = Represents all train lines that can be boarded at this station
           Normally lines are represented by one character.
           LINENAME 456NQR repersents train server for
           4, 5, 6, N, Q, and R trains.

DIVISION = Represents the Line originally the station belonged to BMT, IRT,
           or IND
DATE     = Represents the date (MM-DD-YY)
TIME     = Represents the time (hh:mm:ss) for a scheduled audit event
DESc     = Represent the "REGULAR" scheduled audit event
           (Normally occurs every 4 hours)
           1. Audits may occur more that 4 hours due to planning,
           or troubleshooting activities.
           2. Additionally, there may be a "RECOVR AUD" entry:
           This refers to a missed audit that was recovered.

ENTRIES  = The cumulative entry register value for a device
EXIST    = The cumulative exit register value for a device

In [4]:
import pandas as pd
import dateutil.parser

def date_year_week(date_str):
    parsed_date = dateutil.parser.parse(date_str).isocalendar()
    parsed_date = parsed_date[0] + '-' + parsed_date[1] 
    
def calculate_entry(list_date_entry):
    """
    each item in `list_date_entry` is another list [date, entry]
    """
    list_date_entry.sort(key=lambda x: x[1])
    lowest_entry = list_date_entry[0][1]
    highest_entry = list_date_entry[-1][1]
    #print(list_date_entry)
    return highest_entry - lowest_entry

# Calculate counts for each turnstile per week

columns = ['control_area', 'unit', 'scp', 'station', 'line_name',
            'division', 'date', 'time', 'desc', 'entries', 'exits']
df = pd.read_csv(
       'http://web.mta.info/developers/data/nyct/turnstile/turnstile_170624.txt')
df.columns = columns

test = df[(df.station == '59 ST')
        & ((df.date == '06/17/2017') | (df.date == '06/18/2017'))].sort_values(['time', 'scp'])

test['value_date_entry'] = test.apply(lambda row: [dateutil.parser.parse(row.date  + ' ' + row.time), row.entries], axis=1)
test['index_week'] = test.apply(lambda row: (row.control_area, row.unit, row.scp, row.station, dateutil.parser.parse(row.date)), axis=1)

df_group_week = test.groupby('index_week')['value_date_entry'].apply(list)

# Currently a series, convert to a dataframe
df_group_week = pd.DataFrame({'date_and_entries': df_group_week})
df_group_week['entries_total'] = df_group_week['date_and_entries'].map(calculate_entry)
df_group_week.sample(3)

,date_and_entries,entries_total
index_week,,
"(A002, R051, 02-03-05, 59 ST, 2017-06-18 00:00:00)","[[2017-06-18 00:00:00, 10213678], [2017-06-18 ...",921
"(C022, R212, 01-06-01, 59 ST, 2017-06-17 00:00:00)","[[2017-06-17 00:00:00, 10057603], [2017-06-17 ...",1340
"(A002, R051, 02-03-00, 59 ST, 2017-06-17 00:00:00)","[[2017-06-17 00:00:00, 929806], [2017-06-17 04...",402
